<a href="https://colab.research.google.com/github/villifCoder559/VGGFaceFeatures_PCA/blob/main/VGGFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install faiss-cpu --no-cache

In [3]:
%%capture
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install tensorflow keras_applications
!pip install layer_utils

In [4]:
%%capture
!pip install keras==2.12

In [5]:
import numpy as np
import pandas as pd
import cv2
import keras
from keras_vggface.vggface import VGGFace
from sklearn.datasets import fetch_lfw_people
from skimage.transform import resize
import faiss
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd

<h1>Balancing Dataset

In [29]:
lfw_people = fetch_lfw_people(min_faces_per_person=55, resize=2.4,color=True)
np.random.seed(42)
data=lfw_people.images
target=lfw_people.target
index_list,nr_elements_per_class=np.unique(target,return_counts=True)
min_elements=min(nr_elements_per_class)
balanced_data=[]
balanced_target=[]
for s_class in index_list:
  print(s_class)
  list_indices = np.argwhere(target == s_class).reshape(-1)
  choices = np.random.choice(list_indices, min_elements, replace = False)
  balanced_target.append(np.full(min_elements,s_class))
  balanced_data.append(data[choices])
balanced_data=np.array(balanced_data)
balanced_target=np.array(balanced_target)
balanced_data=balanced_data.reshape(balanced_data.shape[0]*balanced_data.shape[1],balanced_data.shape[2],balanced_data.shape[3],balanced_data.shape[4])
balanced_target=balanced_target.reshape(-1)

0
1
2
3
4
5
6
7
8


In [30]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(faces.images, faces.target, test_size=0.1, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(balanced_data, balanced_target, test_size=0.2, random_state=2)

<h1>Image normalization and standardization

In [8]:
image_samples_train=np.copy(X_train)
image_samples_train=np.array([cv2.resize(img,(224,224)) for img in image_samples_train])
image_samples_train[:,:,:,0]=(image_samples_train[:,:,:,0]-np.mean(X_train[:,:,:,0]))/(np.std(X_train[:,:,:,0]))
image_samples_train[:,:,:,1]=(image_samples_train[:,:,:,1]-np.mean(X_train[:,:,:,1]))/(np.std(X_train[:,:,:,1]))
image_samples_train[:,:,:,2]=(image_samples_train[:,:,:,2]-np.mean(X_train[:,:,:,2]))/(np.std(X_train[:,:,:,2]))

In [9]:
image_samples_test=np.copy(X_test)
image_samples_test=np.array([cv2.resize(img,(224,224)) for img in image_samples_test])
image_samples_test[:,:,:,0]=(image_samples_test[:,:,:,0]-np.mean(X_train[:,:,:,0]))/(np.std(X_train[:,:,:,0]))
image_samples_test[:,:,:,1]=(image_samples_test[:,:,:,1]-np.mean(X_train[:,:,:,1]))/(np.std(X_train[:,:,:,1]))
image_samples_test[:,:,:,2]=(image_samples_test[:,:,:,2]-np.mean(X_train[:,:,:,2]))/(np.std(X_train[:,:,:,2]))

<h1>Extract features

In [27]:
original_result_train.shape

(396, 7, 7, 512)

In [10]:
vggface = VGGFace(model='vgg16',include_top=False,input_shape=(image_samples_train.shape[1],image_samples_train.shape[2],image_samples_train.shape[3]))
result_train=vggface.predict(image_samples_train)
result_test=vggface.predict(image_samples_test)
original_result_train=np.copy(result_train)
original_result_test=np.copy(result_test)

4/4 [==============================] - 2s 577ms/step


<h1>Save/load features from drive

In [77]:
# from google.colab import drive
# import os
# drive.mount('/content/drive')
# drive_path = '/content/drive/MyDrive/'
# # np.save(os.path.join(drive_path, 'faces.npy'), faces)
# np.save(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_train.npy'), original_result_train)
# np.save(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_test.npy'), original_result_test)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# from google.colab import drive
# import os
# drive_path = '/content/drive/MyDrive/'
# result_train=np.load(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_train.npy'),allow_pickle=True)
# result_test=np.load(os.path.join(drive_path, 'VGGFace_features_faces_norm_std_test.npy'),allow_pickle=True)

In [11]:
fixed_components=396
#create reduction (if n_components is negative it means to remove the first n_components instead of lasts)
def create_test_reduction(n_components,result_train,type_index='FlatL2'):
  if type_index not in ['FlatL2', 'FlatIP']:
        raise ValueError("Type must be 'FlatL2' or 'FlatIP'")
  features_reducted=result_train.reshape(result_train.shape[0],-1)
  if n_components > 0:
    tmp_PCA=PCA(n_components=n_components)
    features_reducted=np.ascontiguousarray(tmp_PCA.fit_transform(features_reducted))
  else:
    tmp_PCA=PCA(n_components=fixed_components).fit(features_reducted)
    features_reducted=custom_reduction(tmp_PCA,result_train,n_components)
  faiss.normalize_L2(features_reducted)
  if type_index=='FlatIP':
    return tmp_PCA,features_reducted,faiss.IndexFlatIP(features_reducted.shape[1])
  else:
    return tmp_PCA,features_reducted,faiss.IndexFlatL2(features_reducted.shape[1])

In [12]:
def custom_reduction(pca,result_train,start_index):
  transformed_results = []
  for element in result_train:
    test = element.reshape(1, -1)
    # Apply PCA transformation
    transformed_result = np.dot(test - pca.mean_, pca.components_[(-start_index):].T)
    transformed_results.append(transformed_result)
  return np.array(transformed_results).reshape(result_train.shape[0],pca.components_.shape[0]+start_index)

<h1>Create Tests

In [13]:
list_n_components=[0,20,200,300,-1,-2,-3] #0 -> No PCA, -K -> Without first K eigenvectors, +K -> Use K components
# list_n_components=range(0,-6,-1)
index_type='FlatL2' # FlatIP or FlatL2
pca_objs=[]
features_objs=[]
index_faiss_objs=[]
for n_components in list_n_components:
  if n_components>0:
    pca,features,index=create_test_reduction(n_components,result_train,index_type)
    pca_objs.append(pca)
    features_objs.append(features)
    index_faiss_objs.append(index)
  else:
    if(n_components<0):
      pca,features,index=create_test_reduction(n_components,result_train,index_type)
      pca_objs.append(pca)
      features_objs.append(features)
      index_faiss_objs.append(index)
    else: # dummy PCA
      pca_objs.append(PCA())
      features_objs.append(result_train.reshape(result_train.shape[0],-1))
      if index_type=='FlatIP':
        index_faiss_objs.append(faiss.IndexFlatIP(result_train.reshape(result_train.shape[0],-1).shape[1]))
      else:
        index_faiss_objs.append(faiss.IndexFlatL2(result_train.reshape(result_train.shape[0],-1).shape[1]))
#add vectors to index
count=0
for index in index_faiss_objs:
  print(features_objs[count].shape)
  index.add(features_objs[count])
  count=count+1

(396, 25088)
(396, 20)
(396, 200)
(396, 300)
(396, 395)
(396, 394)
(396, 393)


In [14]:
def compute_precision_recall(ground_truth,target,index_faiss,top_k,reduced_vector):
  TP=-1
  reduced_vector=reduced_vector.reshape(1, -1)
  _,retrived_elements=index.search(reduced_vector, X_train.shape[0])
  retrived_targets_id=ground_truth[retrived_elements]==target
  indices = np.where(retrived_targets_id.reshape(-1))[0]
  if(top_k>len(indices)):
    print('Face nr.',target,'max elements:',len(indices)-1,'(NOT',str(top_k)+')')
    top_k=len(indices)
  stop=indices[top_k-1]+1
  TP=top_k
  total_samples=np.sum(ground_truth==target)
  FP=stop-TP
  FN=total_samples-TP
  precision=0
  if TP+FP!=0:
    precision=TP/(TP+FP)
  recall=TP/((TP+FN))
  return precision,recall

<h1>Compute Precision-recall

In [ ]:
import time
list_top_elements=[100]
i=0
j=0
k=0
precision_results=[]
recall_results=[]
name_results=[]
list_precision_results=[]
list_recall_results=[]
test_vectors=result_test.reshape(result_test.shape[0],-1)
for top_k in list_top_elements:
  for index in index_faiss_objs:
    for vector in test_vectors:
      start=time.time()
      if list_n_components[i] > 0:
        reduced_vector=(vector.reshape(1,-1)-pca_objs[i].mean_)@pca_objs[i].components_.T
      else:
        if list_n_components[i] < 0:
          reduced_vector=(vector.reshape(1,-1)-pca_objs[i].mean_)@pca_objs[i].components_[-list_n_components[i]:].T
        else:
          reduced_vector=vector
      precision,recall=compute_precision_recall(y_train,y_test[j],index,top_k,reduced_vector)
      name_results.append(y_test[j])
      precision_results.append(precision)
      recall_results.append(recall)
      j=j+1
      tmp=time.time()-start
    i=i+1
    j=0
recall_results=np.array(recall_results).reshape(len(list_top_elements),len(index_faiss_objs),test_vectors.shape[0])
precision_results=np.array(precision_results).reshape(len(list_top_elements),len(index_faiss_objs),test_vectors.shape[0])
name_results=np.array(name_results).reshape(len(list_top_elements),len(index_faiss_objs),test_vectors.shape[0])

In [16]:
def create_DataFrame(id_labels,selected_name_result,selected_precision_result,normalization_factors,index_faiss_n_components,list_n_components,index_selected_top_k_retrival):
  str_top_k=str(list_top_elements[index_selected_top_k_retrival])
  text=''
  index_type=''
  precision_vector=np.bincount(selected_name_result,weights=selected_precision_result)/normalization_factors
  recall_vector=np.bincount(selected_name_result,weights=selected_recall_result)/normalization_factors
  F1_vector=2*(precision_vector*recall_vector)/(precision_vector+recall_vector)
  if index_faiss_objs[index_faiss_n_components].__class__.__name__ == 'IndexFlatIP':
    index_type='(IP)'
  else:
    index_type='(L2)'
  if list_n_components[index_faiss_n_components]<0:
    text='/'+str(fixed_components)
  df = pd.DataFrame({
    "Faces":id_labels,
    # ("F"+index_type+"@"+str_top_k+'_'+str(list_n_components[index_faiss_n_components])+text):F1_vector        # F1
    ("P"+index_type+"@"+str_top_k+'_'+str(list_n_components[index_faiss_n_components])+text):precision_vector,  # Precision
    # ("R@"+str_top_k+'_'+str(list_n_components[index_faiss_n_components])):recall_vector,                      # Recall
  })
  return df

In [58]:
df_list=[]
for i in range(precision_results.shape[0]):
  for j in range(precision_results.shape[1]):
    selected_name_result = name_results[i][j]
    selected_recall_result = recall_results[i][j]
    selected_precision_result = precision_results[i][j]
    id_labels,normalization_factors = np.unique(y_test,return_counts=True)
    df=create_DataFrame(id_labels,selected_name_result,selected_precision_result,normalization_factors,j,list_n_components,i)
    df_list.append(df)

F1 Score all retrived<br>
P(IP)@1000_0    -> F-Score(distance_InnerProduct)@all_elements_noPCAReduction<br>
P(IP)@1000_200  ->F-Score(distance_InnerProduct)@all_elements_PCA_200_Components<br>
P(IP)@1000_-2/25-> F-Score(Distance_InnerProduct)@W/h first 2 componets out of 25

In [125]:
merged_df=df_list[0]
for i in range(1,len(df_list)):
  merged_df=pd.merge(merged_df,df_list[i])
merged_df.drop(columns=['Faces']).style.highlight_max(color = 'green', axis = 1).highlight_min(color = 'red', axis = 1)

,P(L2)@10_0,P(L2)@10_-1/396,P(L2)@10_-2/396,P(L2)@10_-3/396,P(L2)@10_-4/396,P(L2)@10_-5/396
0,0.933566,1.000000,0.961433,0.915000,0.887423,0.907947
1,0.495643,0.650642,0.673297,0.641096,0.705854,0.733168
2,0.726794,0.904866,0.921296,0.944444,0.922591,0.595856
3,0.560724,0.721519,0.762788,0.794346,0.673774,0.714881
4,0.779549,0.807582,0.806861,0.838416,0.908702,0.908702
5,0.866898,0.907640,0.877706,0.877039,0.881485,0.834717
6,0.839246,0.945416,0.905483,0.918771,0.905012,0.911640
7,0.981818,0.976923,0.990909,0.976923,0.957576,0.983333
8,0.712858,0.802181,0.794047,0.788607,0.846376,0.842679


In [121]:
merged_df=df_list[0]
for i in range(1,len(df_list)):
  merged_df=pd.merge(merged_df,df_list[i])
merged_df.drop(columns=['Faces']).style.highlight_max(color = 'green', axis = 1).highlight_min(color = 'red', axis = 1)

,P(IP)@10_0,P(IP)@10_-1/396,P(IP)@10_-2/396,P(IP)@10_-3/396,P(IP)@10_-4/396,P(IP)@10_-5/396
0,0.693393,1.000000,0.961433,0.915000,0.887423,0.907947
1,0.953247,0.650642,0.673297,0.641096,0.705854,0.733168
2,0.716182,0.904866,0.921296,0.944444,0.922591,0.595856
3,0.861560,0.721519,0.762788,0.794346,0.673774,0.714881
4,0.423280,0.807582,0.806861,0.838416,0.908702,0.908702
5,0.892608,0.907640,0.877706,0.877039,0.881485,0.834717
6,0.881993,0.945416,0.905483,0.918771,0.905012,0.911640
7,0.905927,0.976923,0.990909,0.976923,0.957576,0.983333
8,0.689644,0.802181,0.794047,0.788607,0.846376,0.842679


In [106]:
merged_df=df_list[0]
for i in range(1,len(df_list)):
  merged_df=pd.merge(merged_df,df_list[i])
merged_df.drop(columns=['Faces']).style.highlight_max(color = 'green', axis = 1).highlight_min(color = 'red', axis = 1)

,P(IP)@10_0,P(IP)@10_10,P(IP)@10_20,P(IP)@10_30,P(IP)@10_40,P(IP)@10_50,P(IP)@10_60,P(IP)@10_70,P(IP)@10_80
0,0.693393,1.000000,1.000000,0.991736,0.991736,0.991736,0.991736,0.991736,0.991736
1,0.953247,0.775601,0.808357,0.813114,0.813489,0.816450,0.825840,0.812635,0.817323
2,0.716182,0.989899,0.942613,0.916486,0.931217,0.925926,0.921296,0.912582,0.908951
3,0.861560,0.764671,0.781259,0.778140,0.773469,0.760409,0.762095,0.764436,0.764902
4,0.423280,0.687605,0.867380,0.879066,0.855940,0.835745,0.833333,0.811127,0.825768
5,0.892608,0.927778,0.921529,0.929487,0.922532,0.930108,0.921911,0.930108,0.931034
6,0.881993,0.918415,0.977273,0.970960,0.970960,0.970960,0.972222,0.972222,0.966880
7,0.905927,1.000000,1.000000,1.000000,0.990909,0.983333,0.990909,0.983333,0.983333
8,0.689644,0.775324,0.839190,0.847099,0.844047,0.837712,0.834633,0.841913,0.850519


In [111]:
merged_df=df_list[0]
for i in range(1,len(df_list)):
  merged_df=pd.merge(merged_df,df_list[i])
merged_df.drop(columns=['Faces']).style.highlight_max(color = 'green', axis = 1).highlight_min(color = 'red', axis = 1)

,P(IP)@10_0,P(IP)@10_-1/25,P(IP)@10_-2/25,P(IP)@10_-3/25,P(IP)@10_-4/25,P(IP)@10_-5/25
0,0.693393,0.991736,0.983471,0.935791,0.915584,0.917726
1,0.953247,0.636873,0.679266,0.627446,0.706392,0.746270
2,0.716182,0.862991,0.955840,0.968254,0.937322,0.572005
3,0.861560,0.742501,0.756341,0.750153,0.661631,0.668222
4,0.423280,0.883418,0.857900,0.900230,0.935638,0.937951
5,0.892608,0.915033,0.881066,0.873219,0.875731,0.825457
6,0.881993,0.941808,0.873585,0.841540,0.834075,0.835618
7,0.905927,0.990909,1.000000,0.976923,0.953846,0.966667
8,0.689644,0.811382,0.794009,0.805772,0.845967,0.862688


In [59]:
merged_df=df_list[0]
for i in range(1,len(df_list)):
  merged_df=pd.merge(merged_df,df_list[i])
merged_df.drop(columns=['Faces']).style.highlight_max(color = 'green', axis = 1).highlight_min(color = 'red', axis = 1)

,P(L2)@100_0,P(L2)@100_20,P(L2)@100_200,P(L2)@100_300,P(L2)@100_-1/396,P(L2)@100_-2/396,P(L2)@100_-3/396
0,0.155623,0.595684,0.377600,0.383791,0.387000,0.268625,0.179912
1,0.124281,0.249043,0.213455,0.221552,0.115779,0.120923,0.122826
2,0.156327,0.364327,0.266565,0.278316,0.280596,0.371817,0.403019
3,0.143845,0.255811,0.193804,0.200298,0.161800,0.166417,0.155882
4,0.129184,0.147289,0.137634,0.137182,0.134989,0.159479,0.171361
5,0.139187,0.344200,0.298148,0.299944,0.225396,0.230262,0.218616
6,0.130889,0.173939,0.168413,0.165498,0.162566,0.158462,0.146085
7,0.172156,0.582893,0.447495,0.451245,0.398425,0.239963,0.209045
8,0.153401,0.363828,0.263263,0.263965,0.207973,0.174017,0.188951


<h1>FisherFace

In [55]:
# count=0
# for pca in pca_objs:
#   print('PCA components',pca_objs[count].n_components_)
#   eigenface_recognizer = cv2.face.EigenFaceRecognizer_create()
#   train=pca.fit_transform(original_result_train.reshape(original_result_train.shape[0],-1))
#   eigenface_recognizer.train(train, np.array(y_train))
#   correct = 0
#   test=pca.transform(original_result_test.reshape(original_result_test.shape[0],-1))
#   total = len(test)
#   for i in range(total):
#       label, confidence = eigenface_recognizer.predict(test[i])
#       if label == y_test[i]:
#           correct += 1
#   accuracy = correct / total * 100
#   print(f"Accuracy: {accuracy}%")
#   print()
#   count+=1

PCA components 396
Accuracy: 93.93939393939394%

PCA components 20
Accuracy: 94.94949494949495%

PCA components 200
Accuracy: 93.93939393939394%

PCA components 300
Accuracy: 91.91919191919192%

PCA components 396
Accuracy: 93.93939393939394%

PCA components 396
Accuracy: 93.93939393939394%

PCA components 396
Accuracy: 93.93939393939394%

